<img src="./images/DLI_Header.png" style="width: 400px;">

# 使用MNIST数据集进行图像分类

在本节中，您将进行深度学习里的“ Hello World”练习：训练深度学习模型以正确地对手写体数字进行分类。

## 目标

在完成本节时，您将：
* 了解深度学习如何解决传统编程方法无法解决的问题
* 了解MNIST手写体数字数据集
* 使用Keras API加载MNIST数据集并准备进行训练
* 创建一个简单的神经网络来执行图像分类
* 使用预先准备的MNIST数据集训练神经网络
* 观察经过训练的神经网络的性能

## 问题：图像分类

在传统的编程中，程序员可以在其代码中说明规则和条件，然后程序可以使用这些规则和条件以正确的方式执行操作。这种方法对很多的问题仍然可以很好地发挥作用。

而图像分类要求程序将以前从未见过的图像正确分类为正确的类别，使用传统编程技术几乎无法解决。程序员如何才能定义规则和条件来正确地对各种各样的图像进行分类，尤其是考虑到他们从未看过的图像？

## 解决方法：深度学习

在很多问题中，为使程序能正确执行所需的规则和条件，人类是难以识别和表达的，而深度学习则擅长解决这样的问题。通过用足够多的数据来训练深层神经网络，并通过训练为网络提供有关其性能的反馈，网络可以通过大量迭代来确定其自身的一组条件，以使其以正确的方式运行。

## MNIST 数据集

在深度学习的历史当中，对[MNIST数据集](http://yann.lecun.com/exdb/mnist/)里的 70000 张手写体数字的图像进行0到9的正确分类是一个重大的进展。如今，这个问题被认为是微不足道的，但是使用MNIST进行图像分类已经成为深度学习的一个“ Hello World”练习。

以下是MNIST数据集中包含的40张图像：

<img src="./images/mnist1.png" style="width: 600px;">

## 训练和测试数据及其标签

在处理用于深度学习的图像时，我们既需要图像本身（通常表示为`X`），也需要这些图像对应的正确的标签（通常表示为`Y`）。 此外，我们都需要一组`X`和`Y`以*训练*模型，然后需要单独的一组`X`和`Y`值以*测试*模型的性能。因此，我们需要将MNIST数据集分为4个部分：

1. `x_train`：用于训练神经网络的图像
2. `y_train`：正确的`x_train`图片标签，用于在训练期间评估模型的预测
3. `x_test`：单独留出来的图像，用来在训练模型后测试模型的性能
4. `y_test`：正确的`x_test`图像标签，用于在训练模型后评估模型的预测

## 用Keras加载数据
目前有许多深度学习框架，每个框架都有各自的优点。在本培训中，我们将使用TensorFlow 2，特别是使用它的开放的[Keras](https://keras.io/) API。 我们为本次培训选择Keras是因为它的API非常适合初学者，并且可以很好地反映您正在学习的术语。它也是在专业的环境中进行深度学习的合理选择，尽管在这方面它并非是唯一的选择，您可能在自己的专业环境中使用了不同的框架。

Keras提供的许多有用功能之一是它包含了用于[许多常见数据集](https://www.tensorflow.org/api_docs/python/tf/keras/datasets)，包括MNIST的诸多辅助功能调用和方法。

我们将从为MNIST加载Keras数据集模块开始：

In [ ]:
from tensorflow.keras.datasets import mnist

借助`mnist`模块，我们可以很容易地加载MNIST数据集，而且它已经划分为训练集和测试集的图像和标签：

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## 探索MNIST数据

上面我们说过，MNIST数据集包含70,000个手写体数字的灰度图像。通过执行以下代码单元，我们可以看到Keras已将60,000张图像分配给训练集，并将10,000张分配给测试集，并且每个图像本身都是一个二维数组，尺寸为28x28：

In [ ]:
x_train.shape

In [ ]:
x_test.shape

此外，我们可以看到这些28x28图像表示为`0`到`255`之间的无符号8位整数的集合，这些值与像素的灰度值相对应，其中“ 0”为黑色，“ 255”为白色，所有其它灰度值介于二者之间：

In [ ]:
x_train.dtype

In [ ]:
x_train.min()

In [ ]:
x_train.max()

In [ ]:
x_train[0]

使用[Matplotlib](https://matplotlib.org/)，我们可以显示数据集中的一个灰度图像：

In [ ]:
import matplotlib.pyplot as plt

image = x_train[0]
plt.imshow(image, cmap='gray')

用这种方法，我们可以看到这是5的28x28像素图像。它是不是3呢？ 答案在`y_train`数据中，其中包含该数据的正确标签。 让我们来看看：

In [ ]:
y_train[0]

## 为训练准备数据

在深度学习中，通常需要将数据转换为理想状态后才能进行训练。对于这个具体的图像分类问题，我们应该对数据执行以下3个任务以准备训练：
1. 展平图像数据，以简化将图像输入模型的过程
2. 归一化图像数据，以使图像的输入值更易于训练模型
3. 对标签进行多分类编码，以使模型更易于使用标签值

### 展平图像数据
尽管深度学习模型可以接受二维图像（本例中为 28x28 像素的图像），但我们仍将简化开始时要用到的数据，将每个图像[重构](https://www.tensorflow.org/api_docs/python/tf/reshape)为 784 个连续像素组成的数组。这个过程也叫展平图像。

在这里，我们使用辅助方法`reshape`来完成此操作：

In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

我们可以确认图像数据已经重构，现在是一维数组的集合，每个数组包含784个像素值：

In [ ]:
x_train.shape

### 对数据进行归一化

深度学习模型处理 0 至 1 之间的数字时更加得心应手（后面会有详细介绍），所以我们会取全部像素值，并用每个值除以 255。我们将此过程称为归一化。我们将在下一讲中更详细地说明为何要执行此操作。

In [ ]:
x_train = x_train / 255
x_test = x_test / 255 

现在我们可以看到，这些值都是介于0.0和1.0之间的浮点值：

In [ ]:
x_train.dtype

In [ ]:
x_train.min()

In [ ]:
x_train.max()

### 多分类编码

如果我们要问，7-2是多少？回答为4比回答为9更接近正确答案。但是，请考虑一下，对于这个图像分类问题，我们并不希望神经网络学习这样的推理，我们只希望它能选择正确的类别，并且知道如果我们有一个数字5的图像，那么猜测4和猜测9是一样糟糕的。

目前，图像的标签是0到9之间的整数。由于这些值表示一个数值范围，因此该模型可能会根据它的猜测值与正确的数值类别的接近程度来判断其性能。

因此，我们将对数据进行称为分类编码的转换。这种转换使每个分类标签都是所有可能类别的集合，其中对应真正类别的那个元素被设置为true。

作为一个简单的示例，假如我们有3个类别：红色，蓝色和绿色。对于给定的颜色所属的类别，其中2个元素将为false，而另一个将为true：

|真正的颜色| 是红色吗？ | 是蓝色吗？ | 是绿色吗？|
|------------|---------|----------|----------|
|红色|True|False|False|
|绿色|False|False|True|
|蓝色|False|True|False|
|绿色|False|False|True|

除了使用“ True”或“ False”，我们还可以使用二进制（0或1）：

|真正的颜色| 是红色吗？ | 是蓝色吗？ | 是绿色吗？|
|------------|---------|----------|----------|
|红色|1|0|0|
|绿色|0|0|1|
|蓝色|0|1|0|
|绿色|0|0|1|

这就是分类编码，它将分类标签的值转换成模型更容易接受的自然而明确的表示形式。因此，如果我们使用下面的值... 

```python
values = ['red, green, blue, green']
```

... 这对神经网络是很难理解的。取而代之的是:

```python
values = [
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 0, 1]
]
```

### 对标签进行多分类编码

Keras提供了一种用于进行分类编码的实用程序，在这里我们使用它对训练集和测试集的标签执行分类编码：

In [ ]:
import tensorflow.keras as keras
num_categories = 10

y_train = keras.utils.to_categorical(y_train, num_categories)
y_test = keras.utils.to_categorical(y_test, num_categories)

这是训练集标签的前10个值，您现在可以看到它们已经被分类编码了：

In [ ]:
y_train[0:9]

## 创建模型

在准备好训练用的数据之后，现在该创建我们的模型了。 第一个基本模型将由几个*层*构成，并分成3个主要部分：

1. 输入层，它将以预先定义好的格式接收数据
2. 几个*隐藏*层，每层由许多*神经元*组成。 每个神经元都有能力通过其权重来影响网络的预测，权重的值将随着网络获得有关其性能的反馈而在多次迭代中进行更新
3. 输出层，它将描述网络对给定图像的预测

### 模型的实例化

首先，我们将使用Keras的`Sequential`模型类创建一个模型实例，该实例将具有一个层序列，数据将依次通过其中的层：

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()

### 创建输入层

接下来，我们将添加输入层。该层将“紧密连接”，这意味着其中的每个神经元及其权重都会影响下一层的每个神经元。为了用Keras做到这一点，我们使用Keras的`Dense`层类。

In [ ]:
from tensorflow.keras.layers import Dense

其中，`units`参数指定该层中神经元的数量。 我们将使用`512`，这是一个很好的起始值。您可以稍后修改此值来查看它如何影响训练。

您将了解有关激励函数的更多信息，但是现在，我们将使用`relu`激励函数。简而言之，它将帮助我们的网络做出比使用线性激励函数更为复杂的预测。

`input_shape`值指定输入数据的形状。在我们的情况下，该数据是784个值的一维数组：

In [ ]:
model.add(Dense(units=512, activation='relu', input_shape=(784,)))

### 创建隐藏层

现在，我们将添加一个额外的密集连接层。同样，稍后将对它们进行更多说明，现在您只需知道这些层为网络提供了更多的参数以有助于其进行预测，亦即它为精确的学习提供了更微妙的机会：

In [ ]:
model.add(Dense(units = 512, activation='relu'))

### 创建输出层

最后，我们将添加一个输出层。该层使用`softmax`激励函数，该函数将使该层的每个输出值是0到1之间的概率值，并且该层的所有输出相加等于1。在这种情况下，由于网络要对一个图像属于0到9中的一个进行预测，所以一共有10个输出，每个输出都会给出模型对图像属于某个类的猜测值（概率）：

In [ ]:
model.add(Dense(units = 10, activation='softmax'))

### 总结模型

Keras提供了模型实例方法`summary`，它将打印出可读的模型摘要：

In [ ]:
model.summary()

### 编译模型

虽然后面还会有更多的细节，但是在实际使用数据训练模型之前，我们需要做的最后一步是对模型进行编译。在这里，我们要指定一个*损失函数*，模型将用该函数来了解其在训练过程中的表现。我们还要指定在模型训练时我们将要跟踪`准确率`：

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

注意，编译模型时可以选择不同的`优化器`，上面的编译使用了默认的优化器`RMSProp`。

## 训练模型

现在，我们已经准备好了训练数据和测试数据，以及一个模型，现在该使用训练数据训练模型，并使用测试数据进行测试了。

“用数据训练模型”通常也称为“使模型拟合数据”。后一种表达方式突显了模型里的权重随时间而变化，以使模型更准确地理解提供给它的数据。

当使用Keras训练模型时，我们使用模型的[`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)方法。它需要以下参数：

* 训练数据
* 训练数据标签
* 它应在整个训练数据集上进行训练的次数（称为*epoch*）
* 验证或测试数据及其标签

运行下面的单元格以训练模型。训练结束后，我们将讨论其输出：

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

### 观察准确率

对于5次训练中的每一次，请注意`accuracy`和`val_accuracy`的得分。`accuracy`说明了模型在所有训练数据上的表现。`val_accuracy`指出了模型在验证数据上的表现。如果您还记得的话，验证数据完全没有用于训练模型。

模型表现得很出色。训练准确率和验证准确率都已快速逼近100%。现在我们便有了可用于检测手写体数字图像并对其进行准确分类的模型。下一步是使用此模型对新的手写数字图像进行分类，我们将此过程称为推理。关于推理流程，我们将在后面的练习中加以探讨。

## 总结

建议您花些时间欣赏一下我们的成果。从过往历史来看，为执行这类任务构建的专家系统都极为复杂，很多从业者花费自己的整个职业生涯来构建这些系统（请查看[官方MNIST页面](http://yann.lecun.com/exdb/mnist/)上的参考资料以及达到各个里程碑的年代）。

MNIST之所以有用，不仅因其对计算机视觉所产生的历史影响，而且它还是一个很好的[基准](http://www.cs.toronto.edu/~serailhydra/publications/tbd-iiswc18.pdf)和调试工具。难以让一种新颖的机器学习体系结构正常工作吗？使用MNIST进行检查。如果该体系结构无法在此数据集上进行学习，则它很也可能无法在更复杂的图像和数据集上进行学习。

### 清理显存
继续前，请执行以下单元清理 GPU 显存。转至下一 notebook 之前需要执行此操作。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

在本节中，您学习了如何构建和训练用于图像分类的一个简单的神经网络。在下一节中，您将构建自己的神经网络并执行数据准备，以解决其他图像分类问题。

请继续启动[02_asl.ipynb](02_asl.ipynb)。